# LET'S COOK
### **This notebook is data exploration notebook**
- Here I will try to explore data and find the insight 
- also will apply some nlp basic 
- stemming and lemmatization and POS as well as NER will be research 
- more .. things might will do while literally exploring the data 

> Please do upvote if you found it helpful 
- Appreciation is a booster for sucess !

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
#importing nlp related libraries 
import nltk
from collections import Counter
from nltk import ngrams
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
recipes = pd.read_csv('/kaggle/input/recipes-from-reddit/Recipes.csv', encoding='latin')
recipes_2 = pd.read_csv('/kaggle/input/recipes-from-reddit/Recipes_2.csv',encoding='latin')
print(recipes.shape)
recipes.head()

In [ ]:
print(recipes_2.shape)
recipes_2.head()


In [ ]:
recipes.append(recipes_2, ignore_index=True)

In [ ]:
# let's print details about the dataset 
print('Details about the dataset')
print(recipes.shape, ':Shape of the dataset')
print('more information about the dataset\n'+str(recipes.info))
print('Description about the dataset\n'+str(recipes.describe()))
print('\nColumns in dateset {}'.format(recipes.columns))

## Exploring column: num_comments

In [ ]:
# using value counts to see what are the occurence of number of comments here 
num_comments =  recipes['num_comments'].value_counts()

In [ ]:
num_comments

In [ ]:
plt.figure(figsize=(20, 8))
recipes['num_comments'].hist(bins=80)
plt.xlabel('Number of comments')
plt.ylabel('Frequency')
plt.title('Number of comments per recipes')
plt.show()


- from the graph it is clear that it is right skewed 
- most of the data comes form 1-25 bin and it was clear by value_counts() 
- max comments is 178 and while most of them have 1 comments only 

# Exploring Column: title

In [ ]:
# let's see first five results for title 
recipes['title'][:5]

- Here I will use some already built functions by me but used in different notebooks 
- these functions will give us the insight of lenght of the sentence 
- will also give number of word and avg number of word 
- and top n-grams plot too (it is fun though! and I really love the out results) :D

In [ ]:
title = recipes['title'] 

In [ ]:
# it will use for ploting character length using histogram 
# This will give us the freq of number of characters len are used in creating the column
def plot_number_of_characters(text, bin_=100, text_=''):
    text_len = text.str.len()
    fig, ax = plt.subplots(1, 1, figsize=(20, 8))
    ax.hist(text_len, color='#341610', bins=bin_)
    ax.set_title(text_, size=22)
    plt.show()
    

In [ ]:
# Number of words in a text column 
# we can examine how the number of words are used for the particular columns
def number_of_words(text, bin_=100, text_=''):
    text_len = text.str.split().map(lambda x : len(x))
    fig, ax = plt.subplots(1, 1, figsize=(20, 8))
    ax.hist(text_len, color='#341610', bins=bin_)
    ax.set_title(text_, size=22)
    plt.show()

In [ ]:
def avg_number_of_words(text, bin_=100, text_=''):
    text_len = text.str.split().map(lambda x : len(x))
    fig, ax = plt.subplots(1, 1, figsize=(20, 8))
    ax.hist(text_len.map(lambda x  : np.mean(x)), color='#341610', bins=bin_)
    ax.set_title(text_, size=22)
    plt.show()

In [ ]:
def clean_text(text):
    
    # Replacing @handle with the word USER
    text_handle = text.str.replace(r'@[\S]+', 'user')
    
    # Replacing the Hast tag with the word hASH
    text_hash = text_handle.str.replace(r'#(\S+)','hash')
    
    # Removing the all the Retweets
    text_r = text_hash.str.replace(r'\brt\b',' ')
    
    # Replacing the URL or Web Address
    text_url = text_r.str.replace(r'((www\.[\S]+)|(http?://[\S]+))','URL')
    
    # Replacing Two or more dots with one
    text_dot = text_url.str.replace(r'\.{2,}', ' ')
    
    # Removing all the special Characters
    text_special = text_dot.str.replace(r'[^\w\d\s]',' ')
    
    # Removing all the non ASCII characters
    text_ascii = text_special.str.replace(r'[^\x00-\x7F]+',' ')
    
    # Removing the leading and trailing Whitespaces
    text_space = text_ascii.str.replace(r'^\s+|\s+?$','')
    
    # Replacing multiple Spaces with Single Space
    Dataframe = text_space.str.replace(r'\s+',' ')
    

    return Dataframe

In [ ]:
# cleaning title 
# we have created cleaning tweet functions since these actually were created on redits it will have tweet like text too 
# so we applying it on the text to clean it.
title = clean_text(title)
title= title.apply(str)


In [ ]:
def n_grams_ploting(text, n=1, grams='unigrams'):
    # Top 20 unigrams for the positive sentiment in the text in whole dataset

    df_data_pos = " ".join(text)
    token_text_pos = word_tokenize(df_data_pos)
    unigrams_pos = ngrams(token_text_pos, n)
    frequency_pos = Counter(unigrams_pos)

    df_pos = pd.DataFrame(frequency_pos.most_common(20))


    # Barplot that shows the top 20 unigrams
    plt.rcParams['figure.figsize'] = [13,9]
    sns.set(font_scale = 1.5, style = 'whitegrid')

    sns_pos_1 = sns.barplot(x = df_pos[1], y = df_pos[0], color = 'lightsteelblue')

    # Setting axes labels
    sns_pos_1.set(xlabel = 'Occurrence', ylabel = grams, title = 'Ngrams plot');
    plt.show()


### so here we have created the helper funtions 
- I will explain the helper functions below 
- n_grams_ploting : it is a helper function to plot top-20 n-grams 
- clean_text() : for cleaning text, its tweet cleaner but we will use it here too 
- number_of_character: to plot number of characters on the plot to see how text differs 
- number of words : to plot number words histo gram 
- avg number of words : histogram to plot avg number of words

In [ ]:
plot_number_of_characters(title, text_='Number of character vs Freq')
number_of_words(title, text_='Number of Words vs Freq')
avg_number_of_words(title, text_='Avg Number of words vs Freq')


In [ ]:
n_grams_ploting(title, n=1, grams='unigrams')
n_grams_ploting(title, n=2, grams='bigrams')
n_grams_ploting(title, n=3, grams='trigrams')

# Exploring Column: comment

In [ ]:
comment = recipes['comment']
comment = clean_text(comment)
comment= comment.apply(str)

In [ ]:
plot_number_of_characters(comment, text_='Number of character vs Freq')
number_of_words(comment, text_='Number of Words vs Freq')
avg_number_of_words(comment, text_='Avg Number of words vs Freq')
n_grams_ploting(comment, n=1, grams='unigrams')
n_grams_ploting(comment, n=2, grams='bigrams')
n_grams_ploting(comment, n=3, grams='trigrams')

# WordCloud 
- Word clouds (also known as text clouds or tag clouds) work in a simple way: the more a specific word appears in a source of textual data (such as a speech, blog post, or database), the bigger and bolder it appears in the word cloud.

- A word cloud is a collection, or cluster, of words depicted in different sizes. The bigger and bolder the word appears, the more often it’s mentioned within a given text and the more important it is.

- Also known as tag clouds or text clouds, these are ideal ways to pull out the most pertinent parts of textual data, from blog posts to databases. They can also help business users compare and contrast two different pieces of text to find the wording similarities between the two.

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib as mpl
stop_words = set(stopwords.words("english"))


In [ ]:
def wordcloud_text(data, column='title'):
                 
                mpl.rcParams['figure.figsize']=(15.0,7.0)    #(6.0,4.0)
                mpl.rcParams['font.size']=12                #10 
                mpl.rcParams['savefig.dpi']=100             #72 
                mpl.rcParams['figure.subplot.bottom']=.1 


                stopwords = set(STOPWORDS)
                

                wordcloud = WordCloud(
                                          background_color='white',
                                          stopwords=stopwords,
                                          max_words=2000,
                                          max_font_size=40, 
                                          random_state=42
                                         ).generate(str(data[column]))

                print(wordcloud)
                fig = plt.figure(1)
                plt.imshow(wordcloud)
                plt.axis('off')
                plt.show()
                fig.savefig("word1.png", dpi=900)

In [ ]:
wordcloud_text(recipes, column='comment')

In [ ]:
wordcloud_text(recipes, column='title')